# Course: Financial Engineering and Risk Management Part 1 by Columbia University
### Implementation of Formulas from Lecture "Interest rates and Fixed Income Intruments"

In [ ]:
import numpy as np

In [112]:
def price_bounds_cashflow(c, rb, rl):
    """
    Calculate the PV bounds on a portfolio given an array of cashflows, a borrowing interest rate, and a lending interest rate
    Assumes the cashflow payouts are once per year
    Assumes the interest rates are annual interest rates
    Assumes no arbitrage
    Assumes you can borrow/lend unlimited amounts at the given interest rates
    Assumes the rates are unchanging throughout the period
    
    Parameters
    ----------
    c : array-like
        Cashflow at the end of each period
    rb: float
        borrowing interest rate
    rl: float
        lending interest rate
    
    Returns
    -------
    lb : float
        Lower bound on the present value of the portfolio of cashflows
    ub : float
        Upper bound on the present value of the portfolio of cashflows
        
    Examples
    --------
    # assume I can borrow an unlimited amount of money at interest rate 3%
    # assume I can lend an unlimited amount of money at interest rate 2%
    # calcluate the price bounds on a portfolio of the following cashflows:
    # $0 at t=0, $100 at t=1, $100 at t=2
    >>> import numpy as np
    >>> c = np.asarray([0,100,100])
    >>> price_bounds_cashflow(c=c, rb=0.03, rl=0.02)
    (191.3469695541521, 194.15609381007306)
    
    # Does not work with vectorized inputs
    """
    c = np.asarray(c)
    if rb < rl: 
        print('borrowing interest rate must be >= lending interest rate')
    lb = np.sum([c_/(1+rb)**k for k, c_ in enumerate(c)])
    ub = np.sum([c_/(1+rl)**k for k, c_ in enumerate(c)])
    
    return (lb, ub)

In [123]:
def price_perpetuity(A, r):
    """
    Calculate price of a perpetuity
    Assuming you can borrow or lend unlimited amounts at the interest rate r
    This is a type of fixed-income security that has default risk, inflation risk, and market risk
    
    Parameters
    ----------
    A : array-like
        Fixed cashflow amount at the end of each period
    r : array-like
        Interest rate
        
    Returns
    -------
    out : array-like
        Present value of a series of perpetuities
        
    Examples
    --------
    # Price of a perpetuity at pays $100/year if you can borrow and lend unlimited amounts at an interest rate of 3.5%
    >>> price_perpetuity(100, 0.035)
    2857.142857142857
    
    # Works with vectorized inputs/outputs
    >>> price_perpetuity([100,200], 0.035)
    [2857.14285714 5714.28571429]
    
    >>> price_perpetuity([100,100], [0.02,0.04])
    array([5000., 2500.])
    
    # same as
    >>> price_perpetuity(100, [0.02,0.04])
    array([5000., 2500.])
    """
    # np.sum([np.divide(A,(1+r)**k) for k in range(np.inf)]) = A/r
    A = np.asarray(A)
    r = np.asarray(r)
    return A/r

In [146]:
def price_annuity(A, r, n):
    """
    Calculate price of an annuity
    Assuming you can borrow or lend unlimited amounts at the interest rate r
    This is a type of fixed-income security that has default risk, inflation risk, and market risk
    
    Annuity = Perpetuity - Perpetuity starting in year n+1
    Annuity price = A/r - (A/r)/(1+r)**n = (A/r)*(1-1/(1+r)**n)
    
    Parameters
    ----------
    A : array-like
        Fixed cashflow amount at the end of year period
    r : array-like
        Interest rate
        
    Returns
    -------
    out : array-like
        Present value of a series of annuities
        
    Examples
    --------
    # price of a 30-year annuity with payments of $100/year while you can borrow/lend money at a 2.5% interest rate
    >>> price_annuity(100, 0.025, 30)
    2093.0292592761148
    
    # Can handle vectorized inputs/outputs
    # Calculate price of two different annuities:
    # first is a 10-year annuity that pays $100/year while you can borrow/lend money at a 2.5% interest rate
    # second is a 30-year annuity that pays $150/year while you can borrow/lend money at a 2.5% interest rate
    >>> price_annuity([100,150], 0.025, [10,30])
    [ 875.2063931  3139.54388891]
    
    
    """
    A = np.asarray(A)
    r = np.asarray(r)
    n = np.asarray(n)
    return (A/r)*(1-(1/(1+r)**n))

In [136]:
def price_bond(F, r, T, ytm):
    """
    Calculate price of a bond
    F is the face value (usually 100 or 1000)
    r is the coupon rate. Pays c=rF/2 every six months
    Maturity T: Number of years until maturity of the bond
    ytm : yield to maturity
    
    Parameters
    ----------
    F
    r
    T
    ytm
    
    Returns
    -------
    out :
        Estimated market price of the bond
    
    Examples
    --------
    # Price of a bond with face value $100, coupon rate of 2%, 5 years to maturity, and a yield to maturity of 1%
    >>> price_bond(100, 0.02, 5, 0.01)
    104.86520593039322
    
    # Vectorized example
    >>> price_bond([100, 1000], 0.02, 5, [0.01, 0.02])
    [ 199.57825124 1009.73041186]
    """
    F = np.asarray(F)
    r = np.asarray(r)
    T = np.asarray(T)
    ytm = np.asarray(ytm)
    
    c = r*F/2
    return np.sum([c/(1+ytm/2)**k for k in np.arange(1, 2*T+1)], axis=0) + F/(1+ytm/2)**(2*T)

### Why do we think in terms of yields?
- Summarizes face value, coupon, maturity, and quality
- Relates to quality: lower quality -> lower price -> higher yield to maturity
- Relates to interest rate movements

But ... yield to maturity is a crude measure. Does not capture everything.

Lower quality means lower price, which means higher yield to maturity.
- Cash payments in the future are going to be discounted with a higher interest rate. Why a higher interest rate? Because I'm not certain that those cash payments are going to come, and therefore I want to discount them very strongly.
- Yield to maturity therefore gives us a way to think about different bonds and compare them. It's a very crude measure because it's trying to summarize four different numbers by a single number

current yield = annualized coupon / current price
YTM = anticipated annual return if you hold a bond until it matures
- accounts for the present value of all of the future coupons of the bond
- need to know market price, par value, coupon interest rate, and time to maturity to calculate YTM
- assumes coupon yields are reinvested at the same rate of the bond's current yield

YTM = (Face Value / Current Value)**(1/n) - 1

The yield to maturity is the annual interest rate at which the current price for the bond P is exactly equal to the present value of the coupon payments plus the face value

So you take all the coupons and you discount them at the rate lambda over 2.

### Verification for price_bond() vectorized

In [142]:
# calcluate the price of two different bonds that pay twice per year:
# First one has a face value of $100, coupon rate of 2%, 5 years to maturity, and a yield to maturity of 1%
# Second one has a face value of $1000, coupon rate of 2%, 5 years to maturity, and a yield to maturity of 2%
# also verify individually that it is producing the correct results
print(price_bond([100, 1000], 0.02, 5, [0.01, 0.02]))
print(price_bond(100, 0.02, 5, 0.01))
print(price_bond(1000, 0.02, 5, 0.02))

[ 104.86520593 1000.        ]
104.86520593039322
1000.0


### Verification for price_annuity() vectorized

In [143]:
# price of a 30-year annuity with payments of $100/year while you can borrow/lend money at a 2.5% interest rate
print(price_annuity(100, 0.025, 30))
print(price_annuity([100,150], 0.025, [10,30]))
print(price_annuity(100, 0.025, 10))
print(price_annuity(150, 0.025, 30))

2093.0292592761148
[ 875.2063931  3139.54388891]
875.2063930970908
3139.5438889141724


### Verification for price_perpetuity() vectorized

In [144]:
# price of a 30-year perpetuity with payments of $100/year while you can borrow/lend money at a 2.5% interest rate
print(price_perpetuity([100,150], 0.025))
print(price_perpetuity(100, 0.025))
print(price_perpetuity(150, 0.025))

[4000. 6000.]
4000.0
6000.0


In [159]:
# 1.) pv of lottery winnings
c = [500000]*20
price_bounds_cashflow(c, 0.1, 0.1)

(4682460.045867208, 4682460.045867208)

In [153]:
# 2.) comparing costs of apartments
print('cost of current apartment', price_bounds_cashflow(c=[1000, 1000, 1000, 1000, 1000, 0], rb=0.12, rl=0.12))
print('cost of new other apartment', price_bounds_cashflow(c=[1800, 900, 900, 900, 900, 0], rb=0.12, rl=0.12))

cost of current apartment (4037.349346626405, 4037.349346626405)
cost of new other apartment (4533.614411963764, 4533.614411963764)


In [168]:
# 3.) d(0,2) given 2 year spot rate = 6.9%
1/(1.069)**2

0.8750736155679097

In [169]:
# 4.) f[1,2] given s1=6.3% and s2=6.9%
(1.069**2)/1.063 -1

0.07503386641580434

In [172]:
"""
5.)
Forward contract on a stock

The current price of a stock is $400 per share and it pays no dividends. 
Assuming a constant interest rate of 8% per year compounded quarterly, what is the stock's theoretical forward price for delivery in 9 months?
"""
400*(1+0.08/4)**3

424.48320000000007

In [171]:
"""
6.)
Bounds using different lending and borrowing rate

Suppose the borrowing rate r_B=10% compounded annually. However,

the lending rate (or equivalently, the interest rate on deposits) is

only 8% compounded annually. Compute the difference between the upper

and lower bounds on the price of an perpetuity that pays A=10,000$ per

year.
"""
price_perpetuity(10000, 0.08) - price_perpetuity(10000, .1)

25000.0

In [170]:
"""
7.)
Value of a Forward contract at an intermediate time

Suppose we hold a forward contract on a stock with expiration 66 months from now. 
We entered into this contract 66 months ago so that when we entered into the contract, the expiration was T = 1 year. 
The stock price 6 months ago was S_0 = 100, the current stock price is 125 and the current interest rate is r = 10%
compounded semi-annually. (This is the same rate that prevailed 66 months ago.) What is the current value of our forward contract?

S[0]=1000
S[0.5]=125
r=10%
F[0] = S[0]*(1+ .1/2)**2 = 110.25
F[0.5] = S[0.5]*(1+ .1/2) = 131.25
f[0.5] = (F[0.5]-F[0])*d(t,T) = 
"""

(131.25-110.25)*(1/(1.05))

20.0

In [176]:
price_perpetuity(27000*4, 0.01858)

array(5812701.82992465)

In [175]:
27000*4

108000

In [178]:
price_annuity(27000*4, 0.01858, 20)

1790395.371077373